<a href="https://colab.research.google.com/github/andrelmsunb/andrelmspuc3/blob/main/MVP3_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. CONFIGURAÇÃO INICIAL E IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("Bibliotecas importadas com sucesso!")


# ==============================================================================
# 2. CARREGAMENTO E DEFINIÇÃO DO PROBLEMA
# ==============================================================================
# Contexto: O objetivo é classificar vinhos como "bons" ou "ruins" com base
# em suas características físico-químicas.

# Carregando o dataset de vinhos tintos do repositório da UCI.
# O ";" é usado como separador neste arquivo.
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, sep=';' )

print("\nAmostra do Dataset Original:")
print(data.head())

# Definição do Problema: Transformando em um problema de classificação binária.
# A qualidade é uma nota de 0 a 10. Vamos definir vinhos com nota >= 7 como "bons" (classe 1)
# e os demais como "ruins" (classe 0).
data['quality_cat'] = np.where(data['quality'] >= 7, 1, 0)

# Separando as features (X) e o alvo (y)
X = data.drop(['quality', 'quality_cat'], axis=1)
y = data['quality_cat']

print("\nDistribuição das classes (0 = Ruim, 1 = Bom):")
print(y.value_counts())


# ==============================================================================
# 3. PREPARAÇÃO DOS DADOS
# ==============================================================================
# 3.1. Separação em Treino e Teste
# Usamos stratify=y para garantir que a proporção de vinhos bons/ruins
# seja a mesma nos conjuntos de treino e teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nFormato dos dados de treino: {X_train.shape}")
print(f"Formato dos dados de teste: {X_test.shape}")

# 3.2. Padronização dos Dados (Opcional, para modelos sensíveis à escala)
# Embora o Random Forest não precise, é uma boa prática ter os dados padronizados
# para comparar com outros modelos, como a Regressão Logística.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nDados padronizados com sucesso.")


# ==============================================================================
# 4. MODELAGEM E TREINAMENTO
# ==============================================================================
# 4.1. Modelo Baseline: Regressão Logística
# Um modelo simples para termos uma base de comparação.
print("\n--- Treinando Modelo Baseline: Regressão Logística ---")
log_reg = LogisticRegression(random_state=42, class_weight='balanced')
log_reg.fit(X_train_scaled, y_train) # Usamos os dados escalados aqui

# 4.2. Modelo Principal: Random Forest Classifier
print("\n--- Treinando Modelo Principal: Random Forest ---")
rf_initial = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
rf_initial.fit(X_train, y_train) # Não precisa de dados escalados

# 4.3. Análise de Feature Importance do Random Forest
print("\nAnalisando a importância das features...")
importances = rf_initial.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values(by='importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df)
plt.title('Importância das Features - Random Forest Inicial')
plt.tight_layout()
plt.show()


# ==============================================================================
# 5. OTIMIZAÇÃO DE HIPERPARÂMETROS (GRIDSEARCHCV)
# ==============================================================================
print("\n--- Otimizando o Random Forest com GridSearchCV ---")
# Definindo a grade de parâmetros para testar
param_grid = {
    'n_estimators': [150, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Configurando o GridSearchCV com validação cruzada (cv=5)
# n_jobs=-1 usa todos os processadores disponíveis para acelerar a busca
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           scoring='f1_weighted', # Métrica de otimização
                           verbose=2)

grid_search.fit(X_train, y_train)

print(f"\nMelhores hiperparâmetros encontrados: {grid_search.best_params_}")

# O melhor modelo já treinado está em grid_search.best_estimator_
rf_optimized = grid_search.best_estimator_


# ==============================================================================
# 6. AVALIAÇÃO FINAL E COMPARAÇÃO
# ==============================================================================
print("\n--- Avaliação Final dos Modelos no Conjunto de Teste ---")

# Fazendo previsões com todos os modelos
y_pred_log_reg = log_reg.predict(X_test_scaled)
y_pred_rf_initial = rf_initial.predict(X_test)
y_pred_rf_optimized = rf_optimized.predict(X_test)

# 6.1. Relatórios de Classificação
print("\nRelatório de Classificação - Regressão Logística:")
print(classification_report(y_test, y_pred_log_reg))

print("\nRelatório de Classificação - Random Forest Inicial:")
print(classification_report(y_test, y_pred_rf_initial))

print("\nRelatório de Classificação - Random Forest Otimizado:")
print(classification_report(y_test, y_pred_rf_optimized))

# 6.2. Tabela Comparativa de Resultados
# Usamos o 'f1-score' com 'weighted avg' para uma comparação justa em dados desbalanceados
results = {
    "Modelo": ["Regressão Logística", "Random Forest Inicial", "Random Forest Otimizado"],
    "Acurácia": [
        accuracy_score(y_test, y_pred_log_reg),
        accuracy_score(y_test, y_pred_rf_initial),
        accuracy_score(y_test, y_pred_rf_optimized)
    ],
    "F1-Score (Ponderado)": [
        float(classification_report(y_test, y_pred_log_reg, output_dict=True)['weighted avg']['f1-score']),
        float(classification_report(y_test, y_pred_rf_initial, output_dict=True)['weighted avg']['f1-score']),
        float(classification_report(y_test, y_pred_rf_optimized, output_dict=True)['weighted avg']['f1-score'])
    ]
}
results_df = pd.DataFrame(results)
print("\n--- Tabela Comparativa de Performance ---\n")
print(results_df.round(3))

# 6.3. Análise de Overfitting do Melhor Modelo
# Comparamos a performance no treino vs. no teste
train_score = rf_optimized.score(X_train, y_train)
test_score = rf_optimized.score(X_test, y_test)

print("\n--- Análise de Overfitting (Modelo Otimizado) ---")
print(f"Pontuação no conjunto de Treino: {train_score:.3f}")
print(f"Pontuação no conjunto de Teste:  {test_score:.3f}")

if train_score > test_score + 0.1:
    print("\nAlerta: Diferença significativa entre treino e teste. Pode haver overfitting.")
else:
    print("\nO modelo parece ter uma boa generalização.")

